### Plots using `plik_lite` data and best-fit values 

In [ ]:
from likelihoods import Planck_plik_lite_likelihood
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [ ]:
# best-fit parameters
#####################
bestfitTT = [3.101, 0.966, 67.30, 0.2652, 0.049242, 0.0831]
bestfitEE = [3.087, 0.968, 69.86, 0.2359, 0.048533, 0.0716]
bestfitTE = [3.112, 0.961, 66.85, 0.2697, 0.049555, 0.0881]

In [ ]:
plTTEE = Planck_plik_lite_likelihood(which="TTEE")

In [ ]:
# get the bes-fit Cls for each models
#####################################
mufac = plTTEE.mufac

plTTEE.get_camb_Cls(bestfitTT)
bfT_TT = plTTEE.bmTT@(mufac*(plTTEE.cmb.cambTCls[30:2509]))
bfE_TT = plTTEE.bmEE@(mufac*(plTTEE.cmb.cambECls[30:1997]))

plTTEE.get_camb_Cls(bestfitEE)
bfT_EE = plTTEE.bmTT@(mufac*(plTTEE.cmb.cambTCls[30:2509]))
bfE_EE = plTTEE.bmEE@(mufac*(plTTEE.cmb.cambECls[30:1997]))

plTTEE.get_camb_Cls(bestfitTE)
bfT_TE = plTTEE.bmTT@(mufac*(plTTEE.cmb.cambTCls[30:2509]))
bfE_TE = plTTEE.bmEE@(mufac*(plTTEE.cmb.cambECls[30:1997]))


In [ ]:
nls = 5

clTTs = plTTEE.cldata[0:nls]
erTTs = np.array([np.sqrt(plTTEE.gcovTT[i,i]) for i in range(nls)])
clEEs = plTTEE.cldata[215:215+nls]
erEEs = np.array([np.sqrt(plTTEE.gcovEE[i,i]) for i in range(nls)])

leffs = plTTEE.leff[0:nls]
lfacs = np.array([leffs[i]*(leffs[i]+1)/(2.*np.pi) for i in range(nls)])

plt.errorbar(leffs, clTTs, erTTs, fmt="o")
plt.plot(leffs, bfT_TE[0:nls])
plt.plot(leffs, bfT_TT[0:nls], linestyle="dotted")
plt.plot(leffs, bfT_EE[0:nls], "--")

#plt.xticks(leffs);
#plt.plot(plTTEE.cldata[215:215+nls])

In [ ]:
plt.errorbar(leffs, clEEs, erEEs, fmt="o")
plt.plot(leffs, bfE_TE[0:nls])
plt.plot(leffs, bfE_TT[0:nls], linestyle="dotted")
plt.plot(leffs, bfE_EE[0:nls], "--")
#plt.xticks(leffs);
#plt.xlim(0, 200)

In [ ]:
# The likelihood-ratio approximation 
####################################
plTTEE.cldata = np.append(bfT_TE, bfE_TE)
maxcom = plTTEE.logLike(bestfitTE)

plTTEE.cldata = np.append(bfT_TT, bfE_EE)
maxsep = plTTEE.logLike(bestfitTE)

print (maxcom, maxsep)

### Simulate some data points with a linear model and obtain $\ln R_{12}$ and its distribution
The model can be simply $y=mx+b$ with say fiducial $m=1$ and $b=0$, and then we can define two experiments that can measure $y$ with some error $\sigma_y(x)$. For the first simple test, we can assume that the data points are uncorrelated; this doesn't mean that the posterior for the model parameters $m,b$ will be uncorrelated. 

In [ ]:
from scipy.optimize import fmin

def ymodel(x, m=1, b=0):
    return m*x + b

def yrealization(x, sigmay=1):
    # return a random realization for y(x)
    return np.random.normal(loc=ymodel(x), scale=sigmay)

In [ ]:
xs1 = range(5)
xs2 = np.arange(-4.5, 0, 1)
xs = np.append(xs2, xs1)
#xs = np.arange(-4.5, 5, 1)
os = np.ones(len(xs1))

lnR12list = []

In [ ]:
# just for single plot
######################
sigma1 = 1
sigma2 = 1.5

d1 = np.array([yrealization(i, sigmay=sigma1) for i in xs1])
d2 = np.array([yrealization(i, sigmay=sigma2) for i in xs2])

def chisq1(params):
    m, b = params
    th1 = np.array([ymodel(x, m=m, b=b) for x in xs1])
    return np.sum(((d1-th1)/sigma1)**2.0)

def chisq2(params):
    m, b = params
    th2 = np.array([ymodel(x, m=m, b=b) for x in xs2])
    return np.sum(((d2-th2)/sigma2)**2.0)

def chisq12(params):
    m, b = params
    return chisq1(params) + chisq2(params)

r1 = fmin(chisq1, x0=[0,1], disp=0);
r2 = fmin(chisq2, x0=[0,1], disp=0);
r12 = fmin(chisq12, x0=[0,1], disp=0);

plt.figure(figsize=(4.5,3.5))

plt.errorbar(xs1, d1, sigma1*os , fmt="go", alpha=0.3, label=r"$\bf{d}_1$")
plt.plot(xs1, np.array([ymodel(x, m=r1[0], b=r1[1]) for x in xs1]), "g--x", label=r"$g_1(\bf{\theta}_1^{\rm ML})$", alpha=0.75)
plt.errorbar(xs2, d2, sigma2*os, fmt="bd", alpha=0.3, label=r"$\bf{d}_2$")
plt.plot(xs2, np.array([ymodel(x, m=r2[0], b=r2[1]) for x in xs2]), "b--x", label=r"$g_2(\bf{\theta}_2^{\rm ML}$)", alpha=0.75)

plt.plot(xs, np.array([ymodel(x, m=r12[0], b=r12[1]) for x in xs]), "kx", linestyle="dotted", alpha=0.5, label=r"$\{g_1({\bf{\theta}}_{12}^{\rm ML}), g_2({\bf{\theta}}_{12}^{\rm ML})\}$")
plt.legend()

plt.xlabel("x")
plt.ylabel("y")
plt.tight_layout()
#plt.style(3)
plt.grid(linestyle="dotted")

print (r1, r2, r12)
#plt.savefig("gtheta.pdf")

In [ ]:
dbfsep1 = np.array([ymodel(x, m=r1[0], b=r1[1]) for x in xs1])
dbfsep2 = np.array([ymodel(x, m=r2[0], b=r2[1]) for x in xs2])

dbfcom1 = np.array([ymodel(x, m=r12[0], b=r12[1]) for x in xs1])
dbfcom2 = np.array([ymodel(x, m=r12[0], b=r12[1]) for x in xs2])

def lnlikesep(params):
        m, b = params
        th1 = np.array([ymodel(x, m=m, b=b) for x in xs1])
        th2 = np.array([ymodel(x, m=m, b=b) for x in xs2])

        return -0.5*((np.sum((dbfsep1-th1)/sigma1)**2.) + np.sum((dbfsep2-th2)/sigma2)**2.)

def lnlikecom(params):
    m, b = params
    th1 = np.array([ymodel(x, m=m, b=b) for x in xs1])
    th2 = np.array([ymodel(x, m=m, b=b) for x in xs2])

    return -0.5*((np.sum((dbfcom1-th1)/sigma1)**2.) + np.sum((dbfcom2-th2)/sigma2)**2.)

lnlikesep(r1)

In [ ]:
import emcee
ndim, nwalkers = 2, 100
sampler_com = emcee.EnsembleSampler(nwalkers, ndim, lnlikecom)
pos_com = [r12+1e-2*np.random.randn(ndim) for i in range(nwalkers)]
%time sampler_com.run_mcmc(pos_com, 1000, );

In [ ]:
samples = sampler_com.chain[:,50:,:].reshape((-1,ndim))
from chainconsumer import ChainConsumer
c = ChainConsumer()
c.add_chain(samples, parameters=["$x$", "$y$"])
fig = c.plotter.plot()

In [ ]:
c.plotter.

In [ ]:
cnt = 0   
sigma1 = 1
sigma2 = 1

while (cnt<1):

    d1 = np.array([yrealization(i, sigmay=sigma1) for i in xs1])
    d2 = np.array([yrealization(i, sigmay=sigma2) for i in xs2])

    def chisq1(params):
        m, b = params
        th1 = np.array([ymodel(x, m=m, b=b) for x in xs1])
        return np.sum(((d1-th1)/sigma1)**2.0)

    def chisq2(params):
        m, b = params
        th2 = np.array([ymodel(x, m=m, b=b) for x in xs2])
        return np.sum(((d2-th2)/sigma2)**2.0)

    def chisq12(params):
        m, b = params
        return chisq1(params) + chisq2(params)

    r1 = fmin(chisq1, x0=[0,1], disp=0);
    r2 = fmin(chisq2, x0=[0,1], disp=0);
    r12 = fmin(chisq12, x0=[0,1], disp=0);

    plt.errorbar(xs1, d1, sigma1*os , fmt="go", alpha=0.3, label="$\textbf{d}_1$")
    plt.plot(xs, np.array([ymodel(x, m=r1[0], b=r1[1]) for x in xs]), "g--")
    plt.errorbar(xs2, d2, sigma2*os, fmt="bd", alpha=0.3)
    plt.plot(xs, np.array([ymodel(x, m=r2[0], b=r2[1]) for x in xs]), "b--")

    plt.plot(xs, np.array([ymodel(x, m=r12[0], b=r12[1]) for x in xs]), "k", alpha=0.2)
    plt.tight_layout()
    plt.legend()
    
    print (r1, r2, r12)

    dbfsep1 = np.array([ymodel(x, m=r1[0], b=r1[1]) for x in xs1])
    dbfsep2 = np.array([ymodel(x, m=r2[0], b=r2[1]) for x in xs2])

    dbfcom1 = np.array([ymodel(x, m=r12[0], b=r12[1]) for x in xs1])
    dbfcom2 = np.array([ymodel(x, m=r12[0], b=r12[1]) for x in xs2])

    # now likelihood functions with the best-fit model
    ##################################################
    def lnlikesep(params):
        m, b = params
        th1 = np.array([ymodel(x, m=m, b=b) for x in xs1])
        th2 = np.array([ymodel(x, m=m, b=b) for x in xs2])

        return -0.5*((np.sum((dbfsep1-th1)/sigma1)**2.) + np.sum((dbfsep2-th2)/sigma2)**2.)

    def lnlikecom(params):
        m, b = params
        th1 = np.array([ymodel(x, m=m, b=b) for x in xs1])
        th2 = np.array([ymodel(x, m=m, b=b) for x in xs2])

        return -0.5*((np.sum((dbfcom1-th1)/sigma1)**2.) + np.sum((dbfcom2-th2)/sigma2)**2.)

    ressep = solve(LogLikelihood=lnlikesep, Prior=Prior, n_dims=2, resume=False,
               sampling_efficiency="model", n_live_points=400)
    rescom = solve(LogLikelihood=lnlikecom, Prior=Prior, n_dims=2, resume=False,
               sampling_efficiency="model", n_live_points=400)

    print (cnt, ressep['logZ'], rescom['logZ'])
    lnR12list.append(ressep['logZ']-rescom['logZ'])
    cnt = cnt + 1

In [ ]:
plt.hist(lnR12list, histtype='step', bins=50);
plt.xlim(-4, 0.5)

In [ ]:
len(np.array(lnR12list)[np.array(lnR12list)<-2])/len(lnR12list)

In [ ]:
import pymultinest
from pymultinest.solve import solve

def Prior(cube):
    return -5+cube*10
    
def Loglike(cube):
    m, b = cube[0], cube[1]
    return -chisq12([m,b])


In [ ]:
ressep = solve(LogLikelihood=lnlikesep, Prior=Prior, n_dims=2, resume=False,
           sampling_efficiency="model", n_live_points=400)
rescom = solve(LogLikelihood=lnlikecom, Prior=Prior, n_dims=2, resume=False,
           sampling_efficiency="model", n_live_points=400)

ressep['logZ'], rescom['logZ']